In [1]:
import pandas as pd
from datasets import load_dataset

ds = load_dataset("huuuyeah/meetingbank")


c:\Users\Nik\anaconda3\envs\dev\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data = ds['train']
test_data = ds['test']
val_data = ds['validation']

ds

DatasetDict({
    train: Dataset({
        features: ['summary', 'uid', 'id', 'transcript'],
        num_rows: 5169
    })
    validation: Dataset({
        features: ['summary', 'uid', 'id', 'transcript'],
        num_rows: 861
    })
    test: Dataset({
        features: ['summary', 'uid', 'id', 'transcript'],
        num_rows: 862
    })
})

In [3]:
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)
val_df = pd.DataFrame(val_data)

In [4]:
train_df.head()

,summary,uid,id,transcript
0,AS AMENDED a bill for an ordinance amending th...,DenverCityCouncil_05012017_17-0161,0,Please refrain from profane or obscene speech....
1,AS AMENDED a bill for an ordinance amending th...,DenverCityCouncil_04102017_17-0161,1,An assessment has called out council bill 161 ...
2,AS AMENDED a bill for an ordinance amending th...,DenverCityCouncil_02272017_17-0161,2,I Please close the voting. Announce the result...
3,Recommendation to respectfully request City Co...,LongBeachCC_03072017_17-0161,3,"Motion passes. Hey, thank you very much. Now w..."
4,AS AMENDED a bill for an ordinance amending th...,DenverCityCouncil_03202017_17-0161,4,All right. Pursuant to Council Bill 3.7 consid...


In [5]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

In [6]:
# Load environment variables.
load_dotenv()

# Set the model name for our LLMs.
OPENAI_MODEL = "gpt-3.5-turbo"
# Store the API key in a variable.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [7]:
from langchain.agents import initialize_agent, load_tools
from langchain.chains.llm import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain

In [9]:
# dropping entries with too many tokens for the model
train_df_sorted = train_df[train_df['transcript'].str.len() <= 16385 * 4].reset_index()
train_df_sorted

,index,summary,uid,id,transcript
0,1,AS AMENDED a bill for an ordinance amending th...,DenverCityCouncil_04102017_17-0161,1,An assessment has called out council bill 161 ...
1,2,AS AMENDED a bill for an ordinance amending th...,DenverCityCouncil_02272017_17-0161,2,I Please close the voting. Announce the result...
2,3,Recommendation to respectfully request City Co...,LongBeachCC_03072017_17-0161,3,"Motion passes. Hey, thank you very much. Now w..."
3,4,AS AMENDED a bill for an ordinance amending th...,DenverCityCouncil_03202017_17-0161,4,All right. Pursuant to Council Bill 3.7 consid...
4,5,AS AMENDED a bill for an ordinance amending th...,DenverCityCouncil_02212017_17-0161,5,"11 Eyes, Resolution 109 has been adopted as am..."
...,...,...,...,...,...
4882,5164,AN ORDINANCE imposing a tax on engaging in the...,SeattleCityCouncil_06052017_CB 118965,5164,Thank you. The bill passed and chair of the Se...
4883,5165,Proclamation Declaring April 8 through April 1...,AlamedaCC_04022019_2019-6703,5165,Okay. Next is a proclamation declaring April e...
4884,5166,Approves an agreement with Kaiser Foundation H...,DenverCityCouncil_01052015_14-1062,5166,I just pulled this one out. This one actually ...
4885,5167,A bill for an Ordinance authorizing and approv...,DenverCityCouncil_06082015_15-0302,5167,The estimated taxable value following redevelo...


In [32]:
train_df_sorted['transcript'][2]

"Motion passes. Hey, thank you very much. Now we're moving on to item 22. Communication from Vice Mayor Richardson recommendation to request City Council approval of the 2017 federal legislative agenda as recommended by the Federal Legislation Committee. There's a motion and a second Vice Mayor. So just a few updates here we have. So we met in November on November 15th to review staff's recommended recommended changes for the 2017 federal legislative agenda. The committee made some fairly substantive changes to the agenda last year when we were reorganized and consolidated some of the statements that may have been repetitive in years past. And because we made so many, so many changes in the last update, and as the new presidential administration gets up and running, the first large committee will be focused on protecting existing revenues that the city current receive currently receives from the federal government, such as the Housing Choice Voucher Program, commonly known as Section e

In [56]:
#audio transcription bot
from openai import OpenAI

client = OpenAI()
audio_file = open("luvvoice.com-20250427-sbtVaC.mp3", 'rb')

transcribed_audio = client.audio.transcriptions.create(
    file=audio_file,
    model='whisper-1',
    response_format='text'
)
transcribed_audio = '"' + transcribed_audio + '"'
print(transcribed_audio)

"Motion passes. Hey, thank you very much. Now we're moving on to item 22. Communication from Vice Mayor Richardson Recommendation to request City Council approval of the 2017 Federal Legislative Agenda as recommended by the Federal Legislation Committee. There's a motion and a second Vice Mayor. So just a few updates here we have. So we met in November on November 15th to review staff's recommended-recommended changes for the 2017 Federal Legislative Agenda. The committee made some fairly substantive changes to the agenda last year when we were reorganized and consolidated some of the statements that may have been repetitive in years past. And because we made so many, so many changes in the last update, and as the new Presidential Administration gets up and running, the first large committee will be focused on protecting existing revenues that the City currently receives from the Federal Government, such as the Housing Choice Voucher Program, commonly known as Section 8, CDBG, other HU

# the following models will take in either the transcribed audio, or a transcribed text document!

In [57]:
#summary bot

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0)


prompt = ChatPromptTemplate.from_messages(
    [("system", "Write a detailed summary of the following:\\n\\n{text}")]
)

# Create Document objects from the filtered transcripts
docs = [Document(page_content=text) for text in transcribed_audio]

# Instantiate chain
chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)

# Invoke chain
try:
    result = chain.invoke({"input_documents": docs})
    print(result['output_text'])
except Exception as e:
    print(f"An error occurred: {e}")

The text is a transcript of a meeting where Vice Mayor Richardson recommended the approval of the 2017 Federal Legislative Agenda as recommended by the Federal Legislation Committee. The meeting took place on November 15th to review the recommended changes for the agenda. The committee made substantial changes to the agenda from the previous year to consolidate and eliminate repetitive statements. The focus was on protecting existing revenues received from the Federal Government, such as the Housing Choice Voucher Program and other HUD programs, as well as supporting workforce programs and mental health services.

The committee also discussed adding language to support federal legislation that would assist in implementing violence prevention programs and homeless prevention. They recommended that the City Council support federal assistance towards establishing substance use treatment facilities in the region. The existing legislative agenda already empowers them to do so.

The committe

In [94]:
# Call to action bot
prompt2 = """You are an intelligent assistant tasked with extracting all unique calls to action from the following text. Return each call to action seperately with a newline character. For example:

-Call to Action 1
-Call to Action 2
-Call to Action 3
...

Here is the text:
"""

# Instantiate chain
chain3 = client.chat.completions.create(
    model=OPENAI_MODEL,
    messages=[{'role': 'system', 'content': prompt2},
              {'role': 'user', 'content': transcribed_audio}]
)

calls = chain3.choices[0].message.content
print(calls)

- Request City Council approval of the 2017 Federal Legislative Agenda
- Support federal legislation for violence prevention programs
- Support federal assistance towards establishing substance use treatment facilities
- Support legislation for mental health services
- Support federal legislation for DOCA program
- Advocate for city's major capital projects
- Continue advocating for gun control, violence prevention, and affordable housing
- Public comment on the agenda
- Motion Passes


In [ ]:
'''# call to action bot

llm2 = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.05)


prompt2 = ChatPromptTemplate.from_messages(
    [("system", "Return calls to action in bullet point form, of the following:\\n\\n{text}")]
)

# Create Document objects from the filtered transcripts
docs = [Document(page_content=text) for text in transcribed_audio]

# Instantiate chain
chain2 = load_summarize_chain(llm, chain_type="stuff", prompt=prompt2)

# Invoke chain
try:
    result2 = chain2.invoke({"input_documents": docs})
    print(result2['output_text'])
except Exception as e:
    print(f"An error occurred: {e}")'''

- Advocate for the City Council to approve the 2017 Federal Legislative Agenda
- Support existing revenue sources from the Federal Government
- Review and implement recommended changes for the 2017 Federal Legislative Agenda
- Focus on protecting existing revenues and supporting workforce programs
- Add language to support mental health services and violence prevention
- Support federal legislation for undocumented immigrants and public safety
- Collect information on individuals or religious affiliations that may harm California's economy
- Advocate for comprehensive and inclusive legislative agenda
- Thank Mayor, Vice Mayor, and Council members for their work
- Ensure funding for major capital projects
- Thank everyone for their efforts and advocacy
- Encourage public comments on the issue
- Acknowledge Councilman's anger
- Motion passes.


In [95]:
meeting_results = {'summary': result['output_text'], 'action calls': calls}

meeting_results['summary'] = 'Summary:\n\n' + meeting_results['summary'] + '\n'
meeting_results['action calls'] = ('Calls to Action:\n' + meeting_results['action calls']).replace('\n', '\n\n')

print(meeting_results['summary'])
print(meeting_results['action calls'])

Summary:

The text is a transcript of a meeting where Vice Mayor Richardson recommended the approval of the 2017 Federal Legislative Agenda as recommended by the Federal Legislation Committee. The meeting took place on November 15th to review the recommended changes for the agenda. The committee made substantial changes to the agenda from the previous year to consolidate and eliminate repetitive statements. The focus was on protecting existing revenues received from the Federal Government, such as the Housing Choice Voucher Program and other HUD programs, as well as supporting workforce programs and mental health services.

The committee also discussed adding language to support federal legislation that would assist in implementing violence prevention programs and homeless prevention. They recommended that the City Council support federal assistance towards establishing substance use treatment facilities in the region. The existing legislative agenda already empowers them to do so.

Th

In [96]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA


results = [meeting_results['summary'], meeting_results['action calls']]

embeddings = SentenceTransformerEmbeddings(model_name='all-mpnet-base-v2')

docsearch = FAISS.from_texts(
    texts=results,
    embedding=embeddings
)

llm3 = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=OPENAI_MODEL, temperature=0)

qa = RetrievalQA.from_chain_type(
    llm=llm3,
    chain_type='stuff',
    retriever=docsearch.as_retriever(search_kwargs={'k':1}),
    return_source_documents=False
)


query = 'What is the summary for this meeting?'

response = qa.invoke(query)
print('Summary:\n')
print(response['result'] + '\n')

query_cta = 'What calls to action were mentioned?'

response_cta = qa.invoke(query_cta + ' Return output in bullet points')
print('Calls to Action\n')
print(response_cta['result'].replace('\n', '\n\n'))

Summary:

The meeting was held to review and recommend changes to the 2017 Federal Legislative Agenda as proposed by the Federal Legislation Committee. The focus was on protecting existing revenues from the Federal Government, supporting workforce programs, mental health services, violence prevention programs, homeless prevention, and substance use treatment facilities. The committee voted to send all recommended changes to the City Council for adoption, including updates necessitated by recent executive orders. The importance of advocating for the city's interests in Washington D.C. and Long Beach initiatives was emphasized, with gratitude expressed for the work done and a commitment to continue advocating for the residents of their communities.

Calls to Action

- Request City Council approval of the 2017 Federal Legislative Agenda

- Support federal legislation for violence prevention programs

- Support federal assistance towards establishing substance use treatment facilities

- S

In [99]:
# NER bot
prompt3 = """You are a smart and intelligent Named Entity Recognition (NER) system. You will extract the unique first and last names from the following text and separate each name with a newline character.

Name 1
Name 2
Name 3
...

Here is the text:
"""

# Instantiate chain
chain3 = client.chat.completions.create(
    model=OPENAI_MODEL,
    messages=[{'role': 'system', 'content': prompt3},
              {'role': 'user', 'content': transcribed_audio}]
)

names = chain3.choices[0].message.content
print(names)

Vice Mayor Richardson
Councilmember Urie
Councilmember Gonzalez
Councilmember Saranga
Councilmember Ringo
Councilmember Pierce


In [100]:
names_list = names.split('\n')
names_list

['Vice Mayor Richardson',
 'Councilmember Urie',
 'Councilmember Gonzalez',
 'Councilmember Saranga',
 'Councilmember Ringo',
 'Councilmember Pierce']

In [101]:
# dummy emails
members_emails = ['vice.richardson@example.org',
'c.urie@gov-office.net',
'gonzalez.council@city-hall.gov',
's.saranga@district.us',
'council.ringo@local-authority.com',
'p.pierce@towncouncil.info']

In [136]:
members_info = pd.DataFrame({'member': names_list, 'email': members_emails})
members_info

,member,email
0,Vice Mayor Richardson,vice.richardson@example.org
1,Councilmember Urie,c.urie@gov-office.net
2,Councilmember Gonzalez,gonzalez.council@city-hall.gov
3,Councilmember Saranga,s.saranga@district.us
4,Councilmember Ringo,council.ringo@local-authority.com
5,Councilmember Pierce,p.pierce@towncouncil.info


In [105]:
members = members_info['members']

# NER bot 2
prompt4 = """You are a smart and intelligent Named Entity Recognition (NER) system. You will identify Unique calls to action and the names of the people the calls to action are directed towards. For example

Name 1: Call to Action
Name 2: Call to Action
Name 3: Call to Action
...

Here is the text:
"""

# Instantiate chain
chain3 = client.chat.completions.create(
    model=OPENAI_MODEL,
    messages=[{'role': 'system', 'content': prompt4},
              {'role': 'user', 'content': transcribed_audio}]
)

names_calls = chain3.choices[0].message.content
print(names_calls)

Vice Mayor Richardson: request City Council approval of the 2017 Federal Legislative Agenda
Councilmember Urie: add language in the Public Safety section to support federal legislation
Mayor: update an existing statement in our agenda supporting the expansion of dockets
Councilman Gonzalez: federal protections
Councilmember Saranga: federal-federal legislation relation trip
Councilmember Ringo: advocating for the city on behalf of Long Beach
Councilmember Pierce: comprehensive advocacy for immigration, gun control, violence prevention, affordable housing


In [113]:
import re

delimiters = [r": ", "\n"]
pattern = "|".join(map(re.escape, delimiters))

names_calls_list = re.split(pattern, names_calls)
names_calls_list

['Vice Mayor Richardson',
 'request City Council approval of the 2017 Federal Legislative Agenda',
 'Councilmember Urie',
 'add language in the Public Safety section to support federal legislation',
 'Mayor',
 'update an existing statement in our agenda supporting the expansion of dockets',
 'Councilman Gonzalez',
 'federal protections',
 'Councilmember Saranga',
 'federal-federal legislation relation trip',
 'Councilmember Ringo',
 'advocating for the city on behalf of Long Beach',
 'Councilmember Pierce',
 'comprehensive advocacy for immigration, gun control, violence prevention, affordable housing']

In [135]:
meeting_actions = {}
i = 0
while i < len(names_calls_list) - 1:
    name = names_calls_list[i].strip()
    action = names_calls_list[i + 1].strip()
    meeting_actions[name] = action
    i += 2

meeting_actions = pd.DataFrame.from_dict([meeting_actions]).T
meeting_actions = meeting_actions.reset_index()
meeting_actions = meeting_actions.rename(columns={'index': 'member', 0: 'action'})
meeting_actions

,member,action
0,Vice Mayor Richardson,request City Council approval of the 2017 Fede...
1,Councilmember Urie,add language in the Public Safety section to s...
2,Mayor,update an existing statement in our agenda sup...
3,Councilman Gonzalez,federal protections
4,Councilmember Saranga,federal-federal legislation relation trip
5,Councilmember Ringo,advocating for the city on behalf of Long Beach
6,Councilmember Pierce,"comprehensive advocacy for immigration, gun co..."
